# Amplifierd API - Profile Management

This notebook demonstrates profile discovery and management operations.

## Overview

Profiles in amplifier define:
- Session configuration (orchestrator settings)
- Provider configurations (LLM services)
- Tool selections
- Hook configurations
- Agent selections
- Context files

**Profile Schema**:
- **Schema v2** (required): The daemon only accepts schema v2 profiles
- Fully resolved profiles with explicit agent and context references
- No runtime inheritance - profiles are self-contained

**Discovery**: Profiles auto-discovered from collections during sync

**Compilation**: Schema v2 profiles compiled to resolve all refs:
- Resolves agent, context, and module references
- Downloads and caches remote assets
- Creates complete profile structure in `share/profiles/{collection}/{profile}/`

**Activation**: Active profile stored in `state/active_profile.txt`

This notebook covers both **read operations** and **write operations**.

In [ ]:
import json

import requests

BASE_URL = "http://127.0.0.1:8420"
API_BASE = f"{BASE_URL}/api/v1"


def print_response(response: requests.Response, title: str = "") -> None:
    if title:
        print(f"\n{'=' * 60}")
        print(f"{title}")
        print(f"{'=' * 60}")
    print(f"Status: {response.status_code} {response.reason}")
    if response.content:
        try:
            data = response.json()
            print(json.dumps(data, indent=2))
            return data
        except json.JSONDecodeError:
            print(response.text)
            return None
    return None


print("✓ Setup complete")

## Read Operations

### List All Profiles

Get all available profiles from all collections:

In [ ]:
response = requests.get(f"{API_BASE}/profiles/")
profiles = print_response(response, "LIST PROFILES")

if profiles:
    print(f"\n✓ Found {len(profiles)} profile(s)")
    for profile in profiles:
        active = "[ACTIVE]" if profile.get("isActive") else ""
        collection = profile.get("collectionId", "unknown")
        print(f"  - {profile['name']} {active}")
        print(f"    Collection: {collection}")
        print(f"    Version: {profile.get('version', 'N/A')}")

### Get Active Profile

Retrieve the currently active profile:

In [ ]:
response = requests.get(f"{API_BASE}/profiles/active")
active_profile = print_response(response, "GET ACTIVE PROFILE")

if active_profile:
    print(f"\n✓ Active profile: {active_profile['name']}")
    print(f"  Collection: {active_profile.get('collectionId', 'unknown')}")
else:
    print("\nℹ No active profile")

### Get Profile Details

Get detailed information about a specific profile:

In [ ]:
# Get details for a profile (change to any available profile name)
profile_name = "default"  # Change this to any available profile

response = requests.get(f"{API_BASE}/profiles/{profile_name}")
profile_details = print_response(response, f"GET PROFILE: {profile_name}")

if profile_details:
    print(f"\n✓ Profile: {profile_details['name']}")
    print(f"  Schema Version: {profile_details.get('schemaVersion', 'unknown')}")
    print(f"  Version: {profile_details.get('version', 'N/A')}")
    print(f"  Description: {profile_details.get('description', 'N/A')}")
    print(f"  Collection: {profile_details.get('collectionId', 'N/A')}")
    print(f"  Active: {profile_details['isActive']}")

    # Show components
    if "session" in profile_details:
        print("\n  Session:")
        if "orchestrator" in profile_details["session"]:
            orch = profile_details["session"]["orchestrator"]
            print(f"    Orchestrator: {orch.get('module', 'N/A')}")
            if "source" in orch:
                print(f"      Source: {orch['source']}")

    agents = profile_details.get("agents", [])
    print(f"\n  Agents: {len(agents)}")
    for agent in agents[:3]:  # Show first 3
        if isinstance(agent, dict):
            print(f"    - {agent.get('ref', agent.get('source', 'unknown'))}")
        else:
            print(f"    - {agent}")

    context = profile_details.get("context", [])
    print(f"\n  Context: {len(context)} ref(s)")
    for ctx in context[:3]:  # Show first 3
        if isinstance(ctx, dict):
            print(f"    - {ctx.get('ref', ctx.get('source', 'unknown'))}")
        else:
            print(f"    - {ctx}")

    print(f"\n  Tools: {len(profile_details.get('tools', []))}")
    print(f"  Providers: {len(profile_details.get('providers', []))}")
    print(f"  Hooks: {len(profile_details.get('hooks', []))}")

## Write Operations

### Activate a Profile

Set a profile as active (activates compiled version if available):

In [ ]:
# Activate a profile (use a profile name from the list above)
profile_to_activate = "default"

response = requests.post(f"{API_BASE}/profiles/{profile_to_activate}/activate")
result = print_response(response, f"ACTIVATE PROFILE: {profile_to_activate}")

if response.ok:
    print(f"\n✓ Activated profile: {result['name']}")
    if result.get("compiled"):
        print("  Using compiled version")
elif response.status_code == 404:
    print(f"\n✗ Profile '{profile_to_activate}' not found")
else:
    print(f"\n✗ Failed to activate profile: {response.status_code}")

### Verify Profile Activation

Confirm the profile is now active:

In [ ]:
response = requests.get(f"{API_BASE}/profiles/active")
active = print_response(response, "VERIFY ACTIVATION")

if active:
    print(f"\n✓ Confirmed: '{active['name']}' is active")
    print(f"  Collection: {active.get('collectionId', 'unknown')}")

### Deactivate Current Profile

Clear the active profile:

In [ ]:
response = requests.delete(f"{API_BASE}/profiles/active")
result = print_response(response, "DEACTIVATE PROFILE")

if response.ok:
    print("\n✓ Profile deactivated")

    # Verify no active profile
    response = requests.get(f"{API_BASE}/profiles/active")
    if response.status_code == 200:
        active = response.json()
        if not active:
            print("✓ Confirmed: No active profile")
    else:
        print("✓ Confirmed: No active profile")

## Profile Schema v2 Format

The daemon only accepts schema v2 profiles. Here's the format:

```yaml
---
profile:
  name: myprofile
  schema_version: 2            # Required!
  version: 1.0.0
  description: "Production profile"

session:
  orchestrator:
    module: loop-streaming
    source: git+https://github.com/org/amplifier-module-loop-streaming@main
    config:
      extended_thinking: true

tools:
  - module: tool-web
    source: git+https://github.com/org/amplifier-module-tool-web@main

agents:
  - git+https://github.com/org/repo@main#subdirectory=agents/researcher.md
  - https://raw.githubusercontent.com/org/repo/main/agents/coder.md
  - /absolute/path/to/agent.md

context:
  - git+https://github.com/org/repo@main#subdirectory=context/
  - /absolute/path/to/context/
---

# Profile description (markdown content)
```

**Key Features**:
- `schema_version: 2` required
- No `extends` field - profiles are fully resolved
- Agents as individual file refs (git+, http, or local paths)
- Context as directory refs
- Support for git subdirectory syntax: `#subdirectory=path`

## Profile Compilation

Profiles are automatically compiled during collection sync. Compilation:
1. Resolves all refs (agents, context, modules)
2. Downloads and caches remote assets
3. Creates complete profile structure

**Compiled Structure**:
```
share/profiles/{collection}/{profile}/
  {profile}.md        # Manifest (preserves original)
  orchestrator/       # Orchestrator module
  agents/             # Agent .md files
    researcher.md
    coder.md
  context/            # Context directories
    README.md
    guidelines.md
  tools/              # Tool modules
  hooks/              # Hook modules
  providers/          # Provider modules
```

**Cache Structure**:
```
cache/git/{commit}/             # Full repo cache
cache/git/{commit}_{subdir}/    # Subdirectory cache
cache/fsspec/http/              # HTTP URL cache
```

## Ref Resolution Examples

Schema v2 profiles support multiple ref formats:

In [ ]:
# Example ref formats
ref_examples = {
    "Git full repo": "git+https://github.com/org/repo@main",
    "Git subdirectory": "git+https://github.com/org/repo@main#subdirectory=agents/",
    "Git specific file": "git+https://github.com/org/repo@v1.0.0#subdirectory=agents/researcher.md",
    "HTTP URL": "https://raw.githubusercontent.com/org/repo/main/agent.md",
    "Local absolute": "/absolute/path/to/agent.md",
    "Local relative": "./relative/path/to/agent.md",
}

print("Supported Ref Formats:")
for name, example in ref_examples.items():
    print(f"\n{name}:")
    print(f"  {example}")

print("\n" + "=" * 60)
print("Cache Behavior:")
print("  git+ refs → cache/git/{commit}/ or cache/git/{commit}_{subdir}/")
print("  http refs → cache/fsspec/http/")
print("  local refs → used directly (no cache)")

## Complete Profile Workflow

Demonstrate a full profile management workflow:

In [ ]:
def profile_workflow():
    """Complete profile management workflow."""

    # 1. List available profiles
    print("1. Listing profiles...")
    response = requests.get(f"{API_BASE}/profiles/")
    if not response.ok:
        print("✗ Failed to list profiles")
        return

    profiles = response.json()
    print(f"✓ Found {len(profiles)} profiles")

    # 2. Get details for first profile
    if profiles:
        profile_name = profiles[0]["name"]
        print(f"\n2. Getting details for '{profile_name}'...")
        response = requests.get(f"{API_BASE}/profiles/{profile_name}")
        if response.ok:
            details = response.json()
            print(f"✓ Collection: {details.get('collectionId', 'unknown')}")
            print(f"✓ Schema version: {details.get('schemaVersion', 'unknown')}")
            print(f"✓ Agents: {len(details.get('agents', []))}")
            print(f"✓ Context: {len(details.get('context', []))}")

        # 3. Activate the profile
        print(f"\n3. Activating '{profile_name}'...")
        response = requests.post(f"{API_BASE}/profiles/{profile_name}/activate")
        if response.ok:
            result = response.json()
            print("✓ Activated successfully")
            if result.get("compiled"):
                print("  Using compiled version")

        # 4. Verify activation
        print("\n4. Verifying activation...")
        response = requests.get(f"{API_BASE}/profiles/active")
        if response.ok:
            active = response.json()
            if active and active["name"] == profile_name:
                print(f"✓ Confirmed: '{profile_name}' is active")

    print("\n✓ Workflow complete")


profile_workflow()

## Summary

### Read Operations
- ✓ List all available profiles (from all collections)
- ✓ Get active profile
- ✓ Get detailed profile information
- ✓ Explore profile components (agents, context, tools, providers, hooks)
- ✓ Schema v2 only - fully resolved profiles

### Write Operations
- ✓ Activate a profile
- ✓ Deactivate current profile
- ✓ **Create profile** (local collection only)
- ✓ **Update profile** (local collection only)
- ✓ **Delete profile** (local collection only, not if active)
- ✓ Error handling for non-existent profiles
- ✓ Permission validation for non-local profiles

## Profile Discovery

**Auto-Discovery**:
- Profiles discovered during collection sync
- Only schema v2 profiles accepted
- Must have `schema_version: 2` in frontmatter
- Must be `*.md` files in collection's `profiles/` directory

**Compiled Structure**:
```
share/profiles/{collection}/{profile}/
  {profile}.md        # Manifest (original preserved)
  orchestrator/       # Orchestrator module
  agents/             # Agent .md files
  context/            # Context directories
  tools/              # Tool modules
  hooks/              # Hook modules
  providers/          # Provider modules
```

## API Endpoints Reference

| Method | Endpoint | Description |
|--------|----------|-------------|
| GET | `/api/v1/profiles/` | List all profiles |
| GET | `/api/v1/profiles/active` | Get active profile |
| GET | `/api/v1/profiles/{name}` | Get profile details |
| POST | `/api/v1/profiles/{name}/activate` | Activate profile |
| DELETE | `/api/v1/profiles/active` | Deactivate profile |
| **POST** | `/api/v1/profiles/` | **Create profile (local only)** |
| **PATCH** | `/api/v1/profiles/{name}` | **Update profile (local only)** |
| **DELETE** | `/api/v1/profiles/{name}` | **Delete profile (local only)** |

## Next Steps

Continue to:
- **04-collection-management.ipynb** - Collection package management
- **05-module-management.ipynb** - Module discovery

In [ ]:
# Try to delete a non-local profile (should fail with 403)
response = requests.delete(f"{API_BASE}/profiles/base")
print(f"Try to delete 'base' profile: {response.status_code}")
if response.status_code == 403:
    print("✓ Correctly blocked: Cannot modify non-local profile")
    print(f"  Error: {response.json()['detail']}")

# Try to delete an active profile (should fail with 409)
# First create and activate a test profile
test_profile = {"name": "test-active", "description": "Test active deletion"}
requests.post(f"{API_BASE}/profiles/", json=test_profile)
requests.post(f"{API_BASE}/profiles/test-active/activate")

response = requests.delete(f"{API_BASE}/profiles/test-active")
print(f"\nTry to delete active profile: {response.status_code}")
if response.status_code == 409:
    print("✓ Correctly blocked: Cannot delete active profile")
    print(f"  Error: {response.json()['detail']}")

# Clean up - deactivate and delete
requests.delete(f"{API_BASE}/profiles/active")
requests.delete(f"{API_BASE}/profiles/test-active")
print("\n✓ Cleanup complete")

### CRUD Restrictions

**Local Collection Only**:
- ✅ Can create/update/delete profiles in `local` collection
- ❌ Cannot modify profiles from other collections (foundation, developer-expertise, etc.)

**Active Profile Protection**:
- ❌ Cannot delete a profile that is currently active
- ✓ Must deactivate first, then delete

**Example Error Handling**:

In [ ]:
# Delete the profile we created
response = requests.delete(f"{API_BASE}/profiles/my-custom-profile")

if response.status_code == 204:
    print("✓ Profile deleted successfully")

    # Verify it's gone
    response = requests.get(f"{API_BASE}/profiles/")
    profiles = response.json()
    profile_names = [p["name"] for p in profiles]

    if "my-custom-profile" not in profile_names:
        print("✓ Confirmed: Profile removed from list")
elif response.status_code == 403:
    print("✗ Cannot delete: Not a local profile")
elif response.status_code == 409:
    print("✗ Cannot delete: Profile is currently active")
elif response.status_code == 404:
    print("✗ Profile not found")
else:
    print(f"✗ Delete failed: {response.status_code}")

### Delete a Local Profile

Remove a profile from the local collection:

In [ ]:
# Update the profile we just created
updates = {
    "description": "Updated: My enhanced custom profile",
    "version": "1.1.0",
    "tools": [
        {"module": "tool-web", "source": "git+https://github.com/microsoft/amplifier-module-tool-web@main"},
        {"module": "tool-search", "source": "git+https://github.com/microsoft/amplifier-module-tool-search@main"},
        {
            "module": "tool-task",
            "source": "git+https://github.com/microsoft/amplifier-module-tool-task@main",
        },  # Added tool
    ],
}

response = requests.patch(f"{API_BASE}/profiles/my-custom-profile", json=updates)
updated = print_response(response, "UPDATE PROFILE")

if response.ok:
    print(f"\n✓ Updated profile: {updated['name']}")
    print(f"  New version: {updated['version']}")
    print(f"  New description: {updated['description']}")
    print(f"  Tools count: {len(updated['tools'])}")
elif response.status_code == 403:
    print("\n✗ Cannot update: Not a local profile")
elif response.status_code == 404:
    print("\n✗ Profile not found")
else:
    print(f"\n✗ Update failed: {response.status_code}")

### Update a Local Profile

Modify an existing profile in the local collection:

In [ ]:
# Create a new profile in local collection
new_profile = {
    "name": "my-custom-profile",
    "version": "1.0.0",
    "description": "My custom profile for testing",
    "providers": [
        {
            "module": "provider-anthropic",
            "source": "git+https://github.com/microsoft/amplifier-module-provider-anthropic@main",
            "config": {"default_model": "claude-sonnet-4"},
        }
    ],
    "tools": [
        {"module": "tool-web", "source": "git+https://github.com/microsoft/amplifier-module-tool-web@main"},
        {"module": "tool-search", "source": "git+https://github.com/microsoft/amplifier-module-tool-search@main"},
    ],
    "hooks": [],
    "orchestrator": {
        "module": "loop-streaming",
        "source": "git+https://github.com/microsoft/amplifier-module-loop-streaming@main",
    },
}

response = requests.post(f"{API_BASE}/profiles/", json=new_profile)
created = print_response(response, "CREATE PROFILE")

if response.status_code == 201:
    print(f"\n✓ Created profile: {created['name']}")
    print(f"  Collection: {created['collectionId']}")
    print(f"  Source: {created['source']}")
    print(f"  Providers: {len(created['providers'])}")
    print(f"  Tools: {len(created['tools'])}")
elif response.status_code == 409:
    print("\nℹ Profile already exists")
else:
    print(f"\n✗ Failed to create profile: {response.status_code}")

### Create a New Profile

Create a custom profile in the local collection: